<a href="https://colab.research.google.com/github/shaileyj/CS175_project/blob/main/stock_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.cluster import AgglomerativeClustering

In [6]:
dfs = []
start = pd.to_datetime('2021-09-30')
end = pd.to_datetime('2022-09-30')

directory = Path('sentiment_stock_data')
for file in directory.iterdir():
  filename = file.name
  stock = filename.split('_')[0]
  filename = "sentiment_stock_data/" + filename
  df = pd.read_csv(filename, parse_dates=['datetime'])
  df = df[['datetime', 'open']]
  df = df[(df['datetime'] >= start) & (df['datetime'] <= end)]
  df = df.sort_values('datetime')
  df = df.rename(columns={'open': stock})
  df = df.set_index('datetime')
  dfs.append(df)

In [7]:
aligned_prices = pd.concat(dfs, axis=1, join="inner")
returns = np.log(aligned_prices).diff().dropna()
corrs = returns.corr()
distances = np.sqrt(2 * (1 - corrs))

In [8]:
n_clusters = 6
clustering = AgglomerativeClustering(
    metric='precomputed',
    linkage='average',
    n_clusters=n_clusters
)
labels = clustering.fit_predict(distances)

In [9]:
clusters = pd.DataFrame({
    'Stock': corrs.index,
    'Cluster': labels
})
clusters

,Stock,Cluster
0,META,5
1,BA,0
2,PG,2
3,NFLX,3
4,MSFT,1
5,AMZN,1
6,AAPL,1
7,GOOG,1
8,COST,4
9,AMD,1
